In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from collections import Counter
import nltk
from nltk.util import ngrams

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train_data = pd.read_csv("/content/drive/MyDrive/Vashantor_CSV_Format/Train/Chittagong Train Translation.csv")
test_data = pd.read_csv("/content/drive/MyDrive/Vashantor_CSV_Format/Test/Chittagong Test Translation.csv")
validation_data = pd.read_csv("/content/drive/MyDrive/Vashantor_CSV_Format/Validation/Chittagong Validation Translation.csv")

In [4]:
train_data.head(5)

,bangla_speech,banglish_speech,chittagong_bangla_speech,chittagong_banglish_speech,region_name,english_speech
0,কেমন আছো ?,kemon acho?,কেন আচো?,Ken acho?,Chittagong,How are you?
1,আজকে আমার মন ভালো নেই,ajke amr mon valo nei,আযিয়া আর মন বালা নাই?,ajiya ar mon bala nai?,Chittagong,I'm not feeling well today
2,তুমি কি করো ?,tumi ki koro?,তুঁই কি গরো?,tui ki goro?,Chittagong,what are you doing?
3,এই গরমে আমার কিছু ভালো লাগে না,ei gorome amar kichu valo lage na,এই গরমত আত্তুন কিচু বালা ন লাগের,ei goromot attun kichu bala no lager,Chittagong,I don't like anything this summer
4,ছেলেটি সাদা রঙয়ের একটি শার্ট পরে এসেছিল,cheleti sada ronger ekti shirt pore eshechilo,ফোয়াইবা সাদা রংওর উজ্ঞা শার্ট ফরি আইস্যিল,foyaiba sada rongor ugga shirt fori aissilo,Chittagong,The boy came wearing a white shirt


In [5]:
train_data.tail(5)

,bangla_speech,banglish_speech,chittagong_bangla_speech,chittagong_banglish_speech,region_name,english_speech
1870,কাউকে মিথ্যা বলে খুশী করার থেকে সত্যি বলে কাঁদ...,kauke mittha bole khushi korar theke sotti bol...,কোনিক্কারে মিথ্যা খইয়েরে কুশী করিবার তুন সত্...,konikkare mittha khoiyere khushi koribar tun...,Chittagong,It's better to be happy for telling a lie than...
1871,"একে অপরের প্রতি যতো বেশী বিশ্বাস থাকবে, তাদের ...","eke oporer proti joto beshi bisshas thakbe, ta...","এইজ্ঞো আরেজ্ঞোর ফ্রতি বোত বিশ্বাস থাইবো, ইতা...","eiggoareggor froti bot bishsas thaibo, itaror ...",Chittagong,"The more trust they have in each other, the he..."
1872,বৃষ্টি বেড়ে তাপমাত্রা কমার সুখবর দিয়েছে আবহা...,brishti bere tapmatra komar sukhobor diyeche a...,বৃষ্টি বাইরলে তাপমাত্রা কমিবার সুখবর দিয়ে আব...,brishti bairle tapmatra komibar sukhobor diye ...,Chittagong,Meteorological office has given the good news ...
1873,পুরো বর্ষা মৌসুমে হাতে গোনা কয়েকদিন মাত্র বৃষ...,puro borsha moushume hate gona koyekdin matro ...,ফুরা বর্ষা মৌসুমোত হাতে গুনা হোয়োজ্ঞা দিন মাত্...,fura borsha mousumot hate guna hoyogga din mat...,Chittagong,After only a handful of days of rain in the en...
1874,খাওয়ার অনেক আগেই যদি ভাত রান্না শেষ হয়ে গিয়ে থ...,khaoyar onek agei jodi vat ranna shesh hoye gi...,খাওয়ার বহুত আগোত যদি ভাত রান্না শেষ ওয় গি তা...,khaoyar bohut agot jodi vat ranna shesh oy gi ...,Chittagong,If the rice is done cooking long before eating...


In [6]:
test_data.head(5)

,bangla_speech,banglish_speech,chittagong_bangla_speech,chittagong_banglish_speech,region_name,english_speech
0,তোমার আব্বু কেমন আছে?,tomar abbu kemon ache?,তোয়ার আব্বু কেন আসে?,toyar abbu ken ase?,Chittagong,How is your father?
1,আমার বড় বোনের আজকে মন ভালো নেই,amar boro boner ajke mon valo nei,আর বর বইনোর আজিয়া মন ভালা নাই,ar bor boinor ajiya mon vala nai,Chittagong,My elder sister is not feeling well today
2,তুমি কি আমাকে এই কাজটি করে দিতে পারবে?,tumi ki amake ei kajti kore dite parbe?,তুইকি আরে হাম্মান গরি দিত্তারিবানা?,tuiki are hamman gori dittaribana?,Chittagong,Can you do this for me?
3,এই গরমে বাহিরে বের হতে ইচ্ছা করে না,ei gorome bahire ber hote iccha kore na,এই গরমত বাইর ওইতো মনত নো সার,ei goromot bair oito monot no sar,Chittagong,Don't want to go out in this summer
4,তোমার সব কথাই ভুল,tomar sob kothai vul,তোয়ার বেক হোতাই ভুল,toyar bek hotai vul,Chittagong,Everything you say is wrong


In [7]:
test_data.tail(5)

,bangla_speech,banglish_speech,chittagong_bangla_speech,chittagong_banglish_speech,region_name,english_speech
370,তুমি কি আমাকে এক গ্লাস পানি এনে দিতে পারবে ?,tumi ki amake ek glass pani ene dite parbe ?,তুই কি আরে এক্কা্কান গ্লাস ফানি আনি দিত্তারিবা...,tui ki are ekkakan glass fani ani dittariba na?,Chittagong,Can you get me a glass of water?
371,আমি পারবো না পানি দিতে,ami parbo na pani dite,আই ন ফাইজ্জুম ফানি দিতাম,ai n faijjum fani ditam,Chittagong,I can't give water
372,তোমার ভাই পড়ালেখাতে অনেক মেধাবি,tomar vai poralekhate onek medhabi,তোয়ার ভাই ফন্নাফরনত বহুত মেধাবি,toyar vai fonnaforonot bohut medhabi,Chittagong,Your brother is very talented in studies
373,আচ্ছা বলো দেখি বাংলাদেশে কয়টি জেলা?,accha bolo dekhi bangladeshe koyti jela?,আচ্ছা হো সাই বাংলাদেশর জেলা হইয়ান?,accha ho sai bangladeshor jela hoiyan?,Chittagong,Well tell me how many districts in Bangladesh?
374,সামনের দিকে যেয়ে মেয়েটি অনেক হাসবে,samner dike jeye meyeti onek hashbe,সামনর মিক্কা যাইয়েরে মাইয়াফোয়া ইবা বহুত হাসিব দে,samnor mikka jaiyere maiyafoya iba bohut hasib de,Chittagong,The girl will smile a lot going forward


In [8]:
validation_data.head(5)

,bangla_speech,banglish_speech,chittagong_bangla_speech,chittagong_banglish_speech,region_name,english_speech
0,বাংলাদেশে ৬৪ টা জেলা,bangladeshe ৬৪ ta jela,বাংলাদেশত ৬৪ ইয়ান জেলা,bangladeshot ৬৪ iyan jela,Chittagong,64 districts in Bangladesh
1,আমরা সবাই গতকাল বাহিরে গিয়েছিলাম,amra sobai gotokal bahire giyechilam,আরা বেয়াক্কুন গতহালিয়া বাইরে গেইলাম,ara beyakkun gotohaliya baire geilam,Chittagong,We all went out yesterday
2,তোমার কথা বলার ধরন অনেক সুন্দর,tomar kotha bolar dhoron onek sundor,তোইয়ার হতা বলার ধরণ বহুত সুন্দর,toiyar hota bolar dhoron bohut sundor,Chittagong,Your way of speaking is very nice
3,বরিশালের মানুষ কেমন হয়?,barishaler manush kemon hoy?,বরিশালর মানুষ হইল্লে অয় দে?,barishalor manush hoille oy de?,Chittagong,How are the people of Barisal?
4,খুলনা জেলা কি অনেক সুন্দর?,khulna jela ki onek sundor?,খুলনা জেলা কি বহুত সুন্দর নাকি?,khulna jela ki bohut sundor naki?,Chittagong,Khulna district is very beautiful?


In [9]:
validation_data.tail(5)

,bangla_speech,banglish_speech,chittagong_bangla_speech,chittagong_banglish_speech,region_name,english_speech
245,তুমি যাও এখান থেকে,tumi jao ekhan theke,তুই যো গয় এরে তুন,tui jo goy ere tun,Chittagong,you go from here
246,আপনি যান এখান থেকে,apni jan ekhan theke,ওরে যন এরে তুন,ore jon ere tun,Chittagong,You go from here
247,তুই যা এখান থেকে,tui ja ekhan theke,তুই যা গয় এরের তুন,tui ja goy erer tun,Chittagong,you go from here
248,আপনারা পার্কে ঘুরে বেড়ান,apnara parke ghure beran,ওনোরা ফার্কত গুরি বেড়াতোক,onora farkot guri beratok,Chittagong,You go for a walk in the park
249,উনি চেষ্টা করেন,uni cheshta koren,ঐইবা চেষ্টা খজ্জিল,oiiba cheshta khojjil,Chittagong,He tries


In [ ]:
!pip install transformers torch pandas

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 2.7 MB/s eta 0:00:00


In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=f199db7d36d225b0cd99c5a0b071e12be17c5823434b25a799f7f368c1e7f32c
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
!pip install sentencepiece

In [ ]:
!pip install transformers[sentencepiece]

In [ ]:
!transformers-cli cache clear

2024-06-13 08:11:30.318975: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-13 08:11:30.319030: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-13 08:11:30.446591: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-13 08:11:30.594619: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-13 08:11:32.070817: W tensorflow/compiler/tf2

In [ ]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 5.1 MB/s eta 0:00:00


In [ ]:
!pip install accelerate -U

In [ ]:
!pip install git+https://github.com/csebuetnlp/normalizer

  Cloning https://github.com/csebuetnlp/normalizer to /tmp/pip-req-build-qhcl6xb5
  Running command git clone --filter=blob:none --quiet https://github.com/csebuetnlp/normalizer /tmp/pip-req-build-qhcl6xb5
  Resolved https://github.com/csebuetnlp/normalizer to commit d405944dde5ceeacb7c2fd3245ae2a9dea5f35c9
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for normalizer: filename=normalizer-0.0.1-py3-none-any.whl size=6859 sha256=c654e2de49df963682d13e3661d2b2367bda6109457556c6590497eb9f4b052d
  Stored in directory: /tmp/pip-ephem-wheel-cache-_biu3yhu/wheels/2e/79/9c/cd96d490298305d51d2da11484bb2c25fd1f759a6906708282
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186459 sha256=28911713f58a0c2ba3c8591211d890bbdc9d87

In [ ]:
!pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 18.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [ ]:
#!pip install transformers==4.10.3

In [ ]:
!pip install torch transformers

In [ ]:
#!pip install accelerate==0.20.3

In [ ]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, DataCollatorForSeq2Seq, Trainer, TrainingArguments
from normalizer import normalize
import pandas as pd
import numpy as np
import nltk
from nltk.translate.bleu_score import corpus_bleu
from sacrebleu import sentence_bleu
from rouge_score import rouge_scorer
from sklearn.model_selection import train_test_split
import os


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
train_data = pd.read_csv("/content/drive/MyDrive/Vashantor_CSV_Format/Train/Chittagong Train Translation.csv")
test_data = pd.read_csv("/content/drive/MyDrive/Vashantor_CSV_Format/Test/Chittagong Test Translation.csv")
validation_data = pd.read_csv("/content/drive/MyDrive/Vashantor_CSV_Format/Validation/Chittagong Validation Translation.csv")

In [11]:
train_data.head()

,bangla_speech,banglish_speech,chittagong_bangla_speech,chittagong_banglish_speech,region_name,english_speech
0,কেমন আছো ?,kemon acho?,কেন আচো?,Ken acho?,Chittagong,How are you?
1,আজকে আমার মন ভালো নেই,ajke amr mon valo nei,আযিয়া আর মন বালা নাই?,ajiya ar mon bala nai?,Chittagong,I'm not feeling well today
2,তুমি কি করো ?,tumi ki koro?,তুঁই কি গরো?,tui ki goro?,Chittagong,what are you doing?
3,এই গরমে আমার কিছু ভালো লাগে না,ei gorome amar kichu valo lage na,এই গরমত আত্তুন কিচু বালা ন লাগের,ei goromot attun kichu bala no lager,Chittagong,I don't like anything this summer
4,ছেলেটি সাদা রঙয়ের একটি শার্ট পরে এসেছিল,cheleti sada ronger ekti shirt pore eshechilo,ফোয়াইবা সাদা রংওর উজ্ঞা শার্ট ফরি আইস্যিল,foyaiba sada rongor ugga shirt fori aissilo,Chittagong,The boy came wearing a white shirt


In [13]:
train_data = train_data.drop(columns=['banglish_speech ','chittagong_banglish_speech ','region_name ','english_speech'],axis=1)
train_data.head()

,bangla_speech,chittagong_bangla_speech
0,কেমন আছো ?,কেন আচো?
1,আজকে আমার মন ভালো নেই,আযিয়া আর মন বালা নাই?
2,তুমি কি করো ?,তুঁই কি গরো?
3,এই গরমে আমার কিছু ভালো লাগে না,এই গরমত আত্তুন কিচু বালা ন লাগের
4,ছেলেটি সাদা রঙয়ের একটি শার্ট পরে এসেছিল,ফোয়াইবা সাদা রংওর উজ্ঞা শার্ট ফরি আইস্যিল


In [14]:
test_data = test_data.drop(columns=['banglish_speech ','chittagong_banglish_speech ','region_name ','english_speech'],axis=1)
test_data.head()

,bangla_speech,chittagong_bangla_speech
0,তোমার আব্বু কেমন আছে?,তোয়ার আব্বু কেন আসে?
1,আমার বড় বোনের আজকে মন ভালো নেই,আর বর বইনোর আজিয়া মন ভালা নাই
2,তুমি কি আমাকে এই কাজটি করে দিতে পারবে?,তুইকি আরে হাম্মান গরি দিত্তারিবানা?
3,এই গরমে বাহিরে বের হতে ইচ্ছা করে না,এই গরমত বাইর ওইতো মনত নো সার
4,তোমার সব কথাই ভুল,তোয়ার বেক হোতাই ভুল


In [15]:
train_data.rename(columns={'bangla_speech ': 'input_text', 'chittagong_bangla_speech '	: 'labels'}, inplace=True)
train_data.head()

,input_text,labels
0,কেমন আছো ?,কেন আচো?
1,আজকে আমার মন ভালো নেই,আযিয়া আর মন বালা নাই?
2,তুমি কি করো ?,তুঁই কি গরো?
3,এই গরমে আমার কিছু ভালো লাগে না,এই গরমত আত্তুন কিচু বালা ন লাগের
4,ছেলেটি সাদা রঙয়ের একটি শার্ট পরে এসেছিল,ফোয়াইবা সাদা রংওর উজ্ঞা শার্ট ফরি আইস্যিল


In [16]:
# Rename the columns to match the expected format
test_data.rename(columns={'bangla_speech ': 'input_text', 'chittagong_bangla_speech '	: 'labels'}, inplace=True)
test_data.head()

,input_text,labels
0,তোমার আব্বু কেমন আছে?,তোয়ার আব্বু কেন আসে?
1,আমার বড় বোনের আজকে মন ভালো নেই,আর বর বইনোর আজিয়া মন ভালা নাই
2,তুমি কি আমাকে এই কাজটি করে দিতে পারবে?,তুইকি আরে হাম্মান গরি দিত্তারিবানা?
3,এই গরমে বাহিরে বের হতে ইচ্ছা করে না,এই গরমত বাইর ওইতো মনত নো সার
4,তোমার সব কথাই ভুল,তোয়ার বেক হোতাই ভুল


In [17]:
validation_data.head()

,bangla_speech,banglish_speech,chittagong_bangla_speech,chittagong_banglish_speech,region_name,english_speech
0,বাংলাদেশে ৬৪ টা জেলা,bangladeshe ৬৪ ta jela,বাংলাদেশত ৬৪ ইয়ান জেলা,bangladeshot ৬৪ iyan jela,Chittagong,64 districts in Bangladesh
1,আমরা সবাই গতকাল বাহিরে গিয়েছিলাম,amra sobai gotokal bahire giyechilam,আরা বেয়াক্কুন গতহালিয়া বাইরে গেইলাম,ara beyakkun gotohaliya baire geilam,Chittagong,We all went out yesterday
2,তোমার কথা বলার ধরন অনেক সুন্দর,tomar kotha bolar dhoron onek sundor,তোইয়ার হতা বলার ধরণ বহুত সুন্দর,toiyar hota bolar dhoron bohut sundor,Chittagong,Your way of speaking is very nice
3,বরিশালের মানুষ কেমন হয়?,barishaler manush kemon hoy?,বরিশালর মানুষ হইল্লে অয় দে?,barishalor manush hoille oy de?,Chittagong,How are the people of Barisal?
4,খুলনা জেলা কি অনেক সুন্দর?,khulna jela ki onek sundor?,খুলনা জেলা কি বহুত সুন্দর নাকি?,khulna jela ki bohut sundor naki?,Chittagong,Khulna district is very beautiful?


In [18]:
validation_data = validation_data.drop(columns=['banglish_speech ','chittagong_banglish_speech ','region_name ','english_speech'],axis=1)
validation_data.head()

,bangla_speech,chittagong_bangla_speech
0,বাংলাদেশে ৬৪ টা জেলা,বাংলাদেশত ৬৪ ইয়ান জেলা
1,আমরা সবাই গতকাল বাহিরে গিয়েছিলাম,আরা বেয়াক্কুন গতহালিয়া বাইরে গেইলাম
2,তোমার কথা বলার ধরন অনেক সুন্দর,তোইয়ার হতা বলার ধরণ বহুত সুন্দর
3,বরিশালের মানুষ কেমন হয়?,বরিশালর মানুষ হইল্লে অয় দে?
4,খুলনা জেলা কি অনেক সুন্দর?,খুলনা জেলা কি বহুত সুন্দর নাকি?


In [19]:
# Rename the columns to match the expected format
validation_data.rename(columns={'bangla_speech ': 'input_text', 'chittagong_bangla_speech '	: 'labels'}, inplace=True)
validation_data.head()

,input_text,labels
0,বাংলাদেশে ৬৪ টা জেলা,বাংলাদেশত ৬৪ ইয়ান জেলা
1,আমরা সবাই গতকাল বাহিরে গিয়েছিলাম,আরা বেয়াক্কুন গতহালিয়া বাইরে গেইলাম
2,তোমার কথা বলার ধরন অনেক সুন্দর,তোইয়ার হতা বলার ধরণ বহুত সুন্দর
3,বরিশালের মানুষ কেমন হয়?,বরিশালর মানুষ হইল্লে অয় দে?
4,খুলনা জেলা কি অনেক সুন্দর?,খুলনা জেলা কি বহুত সুন্দর নাকি?


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [ ]:
import torch
from normalizer import normalize
from transformers import MT5ForConditionalGeneration, AutoTokenizer ,DataCollatorForSeq2Seq, Trainer, TrainingArguments
import os

In [ ]:
model_name = "csebuetnlp/banglat5"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.83k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
class Seq2SeqDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        """
        Args:
            data: A DataFrame containing 'input_text' and 'labels' columns.
            tokenizer: A Hugging Face tokenizer.
            max_length: Maximum sequence length.
        """
        self.input_text = data['input_text'].apply(normalize).tolist()
        self.labels = data['labels'].apply(normalize).tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.input_text)

    def __getitem__(self, idx):
        input_text = self.input_text[idx]
        label_text = self.labels[idx]

        # Tokenize the input text
        input_encodings = self.tokenizer(
            input_text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        # Tokenize the label text to get its 'input_ids' and 'attention_mask'
        label_encodings = self.tokenizer(
            label_text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        return {
            'input_ids': input_encodings['input_ids'].squeeze(),
            'attention_mask': input_encodings['attention_mask'].squeeze(),
            'labels': label_encodings['input_ids'].squeeze(),
        }


In [ ]:
# Create train , test and validation datasets
train_dataset = Seq2SeqDataset(train_data, tokenizer)
test_dataset = Seq2SeqDataset(test_data, tokenizer)
validation_dataset = Seq2SeqDataset(validation_data, tokenizer)

# Create train , test and validation dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)  #batch_size=32
test_dataloader = DataLoader(test_dataset, batch_size=16) #batch_size=32
validation_dataloader = DataLoader(validation_dataset, batch_size=16) #batch_size=32


In [ ]:
# Move the model to the device (CPU or GPU)
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [ ]:
#pip install transformers==4.30


In [ ]:
from transformers import TrainingArguments
from transformers import AdamW

from torch.optim import AdamW

# Create a custom optimizer using torch.optim.AdamW
custom_optimizer = AdamW(
    model.parameters(),
    lr=1e-3,  # Learning rate
    eps=1e-8,  # Epsilon value to prevent division by zero
    weight_decay=0.01,  # Weight decay (L2 regularization)
)

#if you have 1,000 training examples and a batch size of 100, you would have 10 iterations in each epoch (1,000 / 100 = 10)
'''
This parameter determines how many small batches are accumulated before performing a weight update.
In your code, it's set to 8, which means you'll accumulate gradients over 8 small batches before performing a weight update.
This effectively simulates a larger batch size without requiring more GPU memory.
So, you are updating the model's weights less frequently compared to the number of actual batches processed.
'''



'''
Learning rate determines how quickly the model learns from the data.
The learning rate scheduler type is set to "cosine_with_restarts," which is a type of learning rate schedule.
 Warmup steps are the number of initial training steps with a smaller learning rate, and weight decay introduces L2 regularization to the optimizer.
'''



'''
Number of Iterations per Epoch = Number of Training Samples / Batch Size
Total Iterations = Number of Iterations per Epoch * Number of Epochs
'''
# Define the TrainingArguments
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/Movies',
    num_train_epochs=30,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    evaluation_strategy="epoch",
    save_strategy="no",  # Align save strategy with evaluation strategy
    ##save_total_limit=1,
    #save_steps=700,
    learning_rate=1e-3,
    do_train=True,
    do_eval=True,
    remove_unused_columns=False,
    push_to_hub=False,
    report_to="none",
    #load_best_model_at_end=True,
    lr_scheduler_type="cosine_with_restarts",
    weight_decay=0.01,
    logging_dir='/content/drive/MyDrive/Movies',
    logging_steps=10,  # Log every 10 steps,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import DataCollatorForSeq2Seq

# Create a data collator for sequence-to-sequence tasks
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,  # Your Hugging Face tokenizer
    model=model,
    padding=True,
    max_length=128,
    label_pad_token_id=tokenizer.pad_token_id,
)


In [ ]:

# Define the Trainer with the custom optimizer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    optimizers=(custom_optimizer, None),  # Pass the custom optimizer here
)


# **Training start here**

In [ ]:
# Fine-tune the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2717: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/data/data_collator.py:646: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


Epoch,Training Loss,Validation Loss
0,0.705400,0.534248
1,0.552400,0.402178
2,0.487600,0.382440
4,0.426700,0.346980
5,0.203500,0.107591
6,0.099800,0.068662
8,0.043000,0.061919
9,0.037000,0.062603
10,0.024100,0.060951
12,0.014300,0.068387


TrainOutput(global_step=3510, training_loss=0.538030272129488, metrics={'train_runtime': 5037.712, 'train_samples_per_second': 11.166, 'train_steps_per_second': 0.697, 'total_flos': 9609028737564672.0, 'train_loss': 0.538030272129488, 'epoch': 29.936034115138593})

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


In [ ]:
!pip install python-Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 49.6 MB/s eta 0:00:00


In [ ]:
!pip install jiwer

In [ ]:
# Move the model to the device (CPU or GPU)
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [ ]:
!pip install rouge-score
#https://github.com/google-research/google-research/tree/master/rouge
#https://huggingface.co/spaces/evaluate-metric/rouge [Different types of ROUGE scores]

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.7 MB/s eta 0:00:00


# **Loading evaluation metrics**

In [ ]:
import torch
import Levenshtein
from evaluate import load
# Define the move_to_device function
def move_to_device(batch, device):
    if isinstance(batch, torch.Tensor):
        return batch.to(device)
    elif isinstance(batch, list):
        return [move_to_device(item, device) for item in batch]
    elif isinstance(batch, dict):
        return {key: move_to_device(value, device) for key, value in batch.items()}
    else:
        return batch  # If it's not a tensor, list, or dict, leave it as is

# Load the evaluation metric for Character Error Rate (CER) and Word Error Rate (WER) and Exact Match(em)
cer_metric = load("cer")
wer_metric = load("wer")
meteor = load('meteor')
exact_match_metric = load("exact_match")

# Load BLEU and ROUGE metrics
bleu_metric = load("bleu")
rouge_metric = load('rouge')

# Initialize lists to store generated translations and references
generated_translations = []
references = []

# Generate translations for the test dataset
for batch in test_dataloader:
    # Move the batch to CUDA
    batch = move_to_device(batch, 'cuda')

    input_text = batch['input_ids']  # Access the input_text using the correct key
    labels = batch['labels']  # Access the labels using the correct key

    # Generate translations
    translation_ids = model.generate(input_text, max_length=512, num_beams=4, length_penalty=2.0, early_stopping=True)

    # Move the translation_ids to CPU to decode
    translation_ids = translation_ids.to('cpu')

    generated_translation = tokenizer.batch_decode(translation_ids, skip_special_tokens=True)

    generated_translations.extend(generated_translation)
    references.extend(tokenizer.batch_decode(labels, skip_special_tokens=True))  # Decoding the label IDs

# Make sure to move generated_translations back to CPU for evaluation if necessary
generated_translations = [translation if not isinstance(translation, str) else translation for translation in generated_translations]



[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
# Calculate Character Error Rate (CER) and Word Error Rate (WER)
results_CER = cer_metric.compute(predictions=generated_translations, references=references)
results_WER = wer_metric.compute(predictions=generated_translations, references=references)

# Calculate Exact Match (EM) and METEOR(M)
results_em = exact_match_metric.compute(predictions=generated_translations, references=references)
results_met = meteor.compute(predictions=generated_translations, references=references)

# Calculate Bilingual Evaluation Understudy (BLEU) and Recall-Oriented Understudy for Gisting Evaluation (ROUGE)
results_bleu = bleu_metric.compute(predictions=generated_translations, references=references)
results_rouge = rouge_metric.compute(predictions=generated_translations, references=references)


# Calculate Levenshtein Distance
levenshtein_distances = [Levenshtein.distance(generated, reference) for generated, reference in zip(generated_translations, references)]


# **Printing every evaluation metrics**

In [ ]:
print(results_CER)
print(results_WER)
print(results_em)
print(results_met)
print(results_bleu)
print(results_rouge)
print(levenshtein_distances)

0.08414872798434442
0.17894345238095238
{'exact_match': 0.30666666666666664}
{'meteor': 0.8090919261084103}
{'bleu': 0.6256345790494973, 'precisions': [0.8310636731689364, 0.6874466268146883, 0.5693950177935944, 0.4723618090452261], 'brevity_penalty': 0.9992641648602927, 'length_ratio': 0.9992644354542111, 'translation_length': 2717, 'reference_length': 2719}
{'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, 'rougeLsum': 0.0}
[4, 3, 2, 5, 3, 8, 5, 12, 15, 3, 3, 10, 10, 4, 7, 8, 0, 8, 2, 4, 6, 0, 0, 12, 2, 2, 0, 5, 5, 0, 0, 0, 5, 8, 0, 5, 3, 0, 0, 0, 6, 9, 2, 5, 4, 2, 3, 0, 11, 10, 0, 11, 1, 11, 1, 5, 0, 2, 5, 0, 5, 2, 5, 6, 5, 6, 2, 5, 4, 1, 1, 1, 1, 7, 4, 18, 11, 5, 2, 2, 1, 0, 0, 0, 0, 0, 0, 11, 4, 8, 0, 3, 2, 1, 3, 3, 2, 4, 1, 4, 5, 0, 0, 1, 4, 1, 0, 2, 4, 6, 2, 6, 8, 0, 0, 0, 6, 8, 3, 4, 2, 0, 0, 4, 7, 0, 1, 0, 7, 8, 1, 3, 9, 5, 11, 7, 4, 14, 0, 3, 5, 3, 10, 2, 4, 1, 0, 8, 0, 0, 4, 9, 2, 0, 5, 0, 0, 0, 2, 0, 0, 0, 3, 8, 3, 5, 0, 3, 2, 4, 0, 0, 0, 7, 1, 7, 2, 1, 0, 0, 1, 1, 0, 5, 0, 2, 

In [ ]:
"""total_correct = 0
total_samples = len(references)

for generated, reference in zip(generated_translations, references):
    levenshtein_distance = Levenshtein.distance(generated, reference)
    max_length = max(len(generated), len(reference))
    accuracy = 1 - (levenshtein_distance / max_length)
    if accuracy >= 0.8:  # Adjust the threshold as needed
        total_correct += 1

accuracy = total_correct / total_samples
print("Accuracy:", accuracy)


SyntaxError: incomplete input (<ipython-input-54-367bc6da82eb>, line 1)

In [ ]:
"""import torch
from transformers import MT5ForConditionalGeneration, AutoTokenizer
#https://huggingface.co/docs/transformers/model_doc/mt5
model_name = "google/mt5-small" # The variations it has -> mt5-small: 6, mt5-base: 12,mt5-large: 24, mt5-xl: 24, mt5-xxl: 24
model = MT5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
